In [ ]:
%load_ext autoreload
%autoreload 2

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

from eo_tools.util import visualize_S1_products_for_insar

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    producttype="SLC",
    date=("20220110", "20220430"),
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')

## visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.
- Choosing them from the same slice is the easiest though overlapping slices could be combined too.

In [ ]:
visualize_S1_products_for_insar(gdf, gj_aoi)

## download

In [ ]:
# fill indices of products to download
indices_to_download = [8, 13]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')

In [ ]:
# extract paths for processing
file_paths = [item[1]['path'] for item in dl[0].items()]